In [17]:
!pip install hmmlearn
import hmmlearn,hmmlearn.hmm
import numpy as np
import nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
temp_file = open('/content/drive/MyDrive/Text-mining/hmm_sentences.txt','r', encoding='utf-8', errors='ignore')
text_lines = temp_file.readlines()

In [19]:
len(text_lines)

1000

In [20]:
hmm_sentences = []
for line in text_lines:
    temp_text = line.strip()
    temp_text = ' '.join(temp_text.split())
    hmm_sentences.append(temp_text)


In [21]:
hmm_sentences[0]

'where may you feed ?'

In [24]:
nltk.download('punkt')
# Tokenize downloaded texts and change them to NLTK format
mydownloaded_nltk_texts = []
for k in range(len(hmm_sentences)):
    temp_tokenizedtext = nltk.word_tokenize(hmm_sentences[k])
    temp_nltktext = nltk.Text(temp_tokenizedtext)
    mydownloaded_nltk_texts.append(temp_nltktext)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [25]:
len(mydownloaded_nltk_texts)

1000

In [26]:
# Make all downloaded texts lowercase
mydownloaded_lowercase_texts = []
for k in range(len(mydownloaded_nltk_texts)):
    temp_lowercase_text = []
    for l in range(len(mydownloaded_nltk_texts[k])):
        lowercase_word = mydownloaded_nltk_texts[k][l].lower()
        temp_lowercase_text.append(lowercase_word)
    temp_lowercasetest = nltk.Text(temp_lowercase_text)
    mydownloaded_lowercase_texts.append(temp_lowercase_text)

In [27]:
len(mydownloaded_lowercase_texts)

1000

In [28]:
# Convert a POS tag for WordNet
def tagtowordnet(postag):
    wordnettag=-1
    if postag[0]=='N':
        wordnettag='n'
    elif postag[0]=='V':
        wordnettag='v'
    elif postag[0]=='J':
        wordnettag='a'
    elif postag[0]=='R':
        wordnettag='r'
    return(wordnettag)


In [29]:
# POS tag and lemmatize the loaded texts
# Download tagger and wordnet resources if you do not have them already
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

# lemmatize downloaded texts
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatizetext(nltktexttolemmatize):
    # tag the text with POS tags
    taggedtext = nltk.pos_tag(nltktexttolemmatize)
    # lemmatize each word text
    lemmatizedtext = []
    for l in range(len(taggedtext)):
        # Lemmatize a word using the WordNet converted POS tag
        wordtolemmatize = taggedtext[l][0]
        wordnettag = tagtowordnet(taggedtext[l][1])
        if wordnettag != -1:
            lemmatizedword = lemmatizer.lemmatize(wordtolemmatize, wordnettag)
        else:
            lemmatizedword = wordtolemmatize
        # store the lemmatized word
        lemmatizedtext.append(lemmatizedword)
    return(lemmatizedtext)


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [31]:
nltk.download('omw-1.4')
mydownloaded_lemmatizedtexts = []
for k in range(len(mydownloaded_lowercase_texts)):
    lemmatizedtext = lemmatizetext(mydownloaded_lowercase_texts[k])
    lemmatizedtext = nltk.Text(lemmatizedtext)
    mydownloaded_lemmatizedtexts.append(lemmatizedtext)


[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [32]:
mydownloaded_lemmatizedtexts[0]

<Text: where may you fee ?...>

In [33]:
# find the vocabularies of each document
myvocabularies = []
myindices_in_vocabularies = []
for k in range(len(mydownloaded_lemmatizedtexts)):
    # get unique words and where they occur
    temptext = mydownloaded_lemmatizedtexts[k]
    unique_results = np.unique(temptext, return_inverse = True)
    unique_words = unique_results[0]
    word_indices = unique_results[1]
    # store the vocabularies and the indices
    myvocabularies.append(unique_words)
    myindices_in_vocabularies.append(word_indices)

In [34]:
len(myvocabularies), len(myindices_in_vocabularies)

(1000, 1000)

In [35]:
# unify the vocabularies
# first concatenate all vocabularies
tempvocabulary = []
for k in range(len(mydownloaded_lemmatizedtexts)):
    tempvocabulary.extend(myvocabularies[k])

# find unique words among all the vocabularies
uniqueresults = np.unique(tempvocabulary, return_inverse = True)
unifiedvocabulary = uniqueresults[0]
wordindices = uniqueresults[1]


In [36]:
len(unifiedvocabulary)

43

In [37]:
# translate previous indices to the unified vocabulary
# must keep track where each vocabulary started in the concatenated one
vocabularystart = 0
myindices_in_unifiedvocabulary = []
for k in range(len(mydownloaded_lemmatizedtexts)):
    # in order to shift word indices, we must temporarily
    # convert their data type to a numpy array
    tempindices = np.array(myindices_in_vocabularies[k])
    tempindices = tempindices + vocabularystart
    tempindices = wordindices[tempindices]
    myindices_in_unifiedvocabulary.append(tempindices)
    vocabularystart = vocabularystart + len(myvocabularies[k])


In [38]:
len(myindices_in_unifiedvocabulary)

1000

In [39]:
wordindices[np.array(myindices_in_vocabularies[0])]

array([38, 27, 42, 15,  2])

In [40]:
wordindices

array([ 2, 15, 27, ..., 37, 40, 41])

In [41]:
unifiedvocabulary


array([',', '.', '?', 'a', 'and', 'answer', 'ask', 'be', 'bird', 'but',
       'can', 'cat', 'cautious', 'dog', 'explain', 'fee', 'feed', 'fox',
       'friendly', 'he', 'help', 'her', 'him', 'how', 'however', 'i',
       'insightful', 'may', 'me', 'robot', 'she', 'small', 'strong',
       'the', 'them', 'then', 'they', 'when', 'where', 'why', 'will',
       'wise', 'you'], dtype='<U10')

In [42]:
unifiedvocabulary[myindices_in_unifiedvocabulary[0]]

array(['where', 'may', 'you', 'fee', '?'], dtype='<U10')

In [43]:
concatenated_data = []
documentlengths = []
for k in range(len(myindices_in_unifiedvocabulary)):
    concatenated_data.extend(myindices_in_unifiedvocabulary[k])
    documentlengths.append(len(myindices_in_unifiedvocabulary[k]))
    
concatenated_data = np.matrix(concatenated_data).T

In [45]:
myhmm = hmmlearn.hmm.CategoricalHMM(n_components=5, n_iter=100, verbose=True)
myhmm_fitted = myhmm.fit(concatenated_data, lengths = documentlengths)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
         1      -70111.5311             +nan
         2      -60140.1392       +9971.3918
         3      -59639.0542        +501.0851
         4      -59172.1331        +466.9211
         5      -58695.1087        +477.0244
         6      -58128.5592        +566.5495
         7      -57442.5450        +686.0141
         8      -56734.1830        +708.3620
         9      -56165.7918        +568.3913
        10      -55770.8847        +394.9070
        11      -55469.4782        +301.4066
        12      -55198.4898        +270.9884
        13      -54916.0775        +282.4124
        14      -54610.7973        +305.2801
        15      -54277.3450        +333.4523
       

In [46]:
myhmm_fitted.startprob_

array([1.51385396e-48, 0.00000000e+00, 6.33566173e-01, 1.15378995e-25,
       3.66433827e-01])

In [47]:
myhmm_fitted.startprob_.shape

(5,)

In [48]:
myhmm_fitted.emissionprob_


array([[3.20126346e-061, 2.45224376e-022, 4.22302390e-009,
        1.66759381e-003, 9.62693473e-048, 8.08860998e-043,
        3.69635037e-053, 3.01395459e-001, 8.39362937e-002,
        0.00000000e+000, 5.55436668e-029, 2.54141788e-002,
        1.86311749e-002, 3.49485564e-002, 3.35641539e-047,
        1.30266391e-057, 8.08345315e-027, 9.06300351e-002,
        2.72165839e-002, 1.31642957e-002, 3.00315173e-044,
        5.15659504e-127, 2.18088934e-097, 1.67757914e-203,
        0.00000000e+000, 7.32657817e-002, 5.27674485e-002,
        1.64382707e-028, 2.76575762e-050, 8.10244042e-002,
        7.18540571e-003, 2.03805524e-002, 6.45983739e-002,
        3.23712731e-004, 1.64744478e-103, 7.87056044e-213,
        2.78313997e-002, 1.15567459e-195, 2.67288765e-194,
        3.83923955e-193, 2.83250297e-028, 2.32574274e-002,
        5.23613181e-002],
       [0.00000000e+000, 4.92630943e-003, 2.30081633e-002,
        1.95957542e-186, 7.67591452e-008, 5.20969571e-004,
        5.88979593e-009, 6.163

In [49]:
np.sum(myhmm_fitted.emissionprob_, axis=1)

array([1., 1., 1., 1., 1.])

In [50]:
myhmm_fitted.emissionprob_.shape

(5, 43)

In [51]:
print(myhmm_fitted.transmat_)

[[1.80137450e-01 6.85881283e-01 2.41863111e-69 7.02821587e-02
  6.36991084e-02]
 [1.12588724e-01 8.27504095e-09 3.07441034e-48 1.62537656e-22
  8.87411268e-01]
 [9.25450483e-01 1.00072423e-13 7.45495167e-02 7.13975895e-15
  6.23068343e-34]
 [2.66066524e-26 4.42943486e-68 2.05477298e-01 4.66857045e-01
  3.27665657e-01]
 [1.70529533e-24 1.42423060e-62 2.28356957e-01 7.71643043e-01
  1.55400295e-15]]


In [52]:
np.sum(myhmm_fitted.transmat_, axis=1)

array([1., 1., 1., 1., 1.])

In [53]:
myhmm_fitted.transmat_.shape

(5, 5)